In [1]:
import pandas as pd
import numpy as np

In [2]:
old = pd.read_csv("tweet_emotions.csv")
old.shape

(40000, 3)

In [3]:
old['sentiment'].unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

In [4]:
def sorter(emotion):
  val = emotion
  if emotion == 'empty':
    val = 'sadness'
  elif emotion == 'enthusiasm':
    val = 'joy'
  elif emotion == 'worry':
    val = 'fear'
  elif emotion == 'fun':
    val = 'joy'
  elif emotion == 'hate':
    val = 'anger'
  elif emotion == 'happiness':
    val = 'joy'
  elif emotion == 'boredom':
    val = 'neutral'
  elif emotion == 'relief':
    val = 'joy'
  return val


In [5]:
old['relabeled'] = old['sentiment'].apply(sorter)

In [6]:
old['relabeled'].unique()

array(['sadness', 'joy', 'neutral', 'fear', 'surprise', 'love', 'anger'],
      dtype=object)

In [8]:
old = old.drop_duplicates('content', keep=False)

In [9]:
old.groupby('relabeled').agg("count")

,tweet_id,sentiment,content
relabeled,,,
anger,1432,1432,1432
fear,8421,8421,8421
joy,9216,9216,9216
love,3770,3770,3770
neutral,8759,8759,8759
sadness,5963,5963,5963
surprise,2177,2177,2177


In [10]:
old.shape

(39738, 4)

In [11]:
to_mix = old[['content', 'relabeled']]
to_mix['emotions'] = to_mix['relabeled']
to_mix['text'] = to_mix['content']
to_mix = to_mix.drop(['content', 'relabeled'],axis=1)
to_mix.head()

<ipython-input-11-e2884f61a9fc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_mix['emotions'] = to_mix['relabeled']


,emotions,text
0,sadness,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,joy,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [12]:
from sklearn.model_selection import train_test_split

# a function to split data (doing this a lot in this notebook)
def data_splitter(data_df, test_pct, train_pct):
  return train_test_split(data_df['text'], data_df['emotions'],
                                                      test_size=test_pct,
                                                      train_size=train_pct,
                                                      stratify=data_df['emotions'],
                                                      shuffle=True,
                                                      random_state=11) # make sure this is reliable

In [13]:
#renamed columns above and now we split the "dirty" set in to training and test
X_trainmix, X_valmix, y_trainmix, y_valmix = data_splitter(to_mix, test_pct=0.1, train_pct=0.9)

In [14]:
#split into train and test
mix_train = pd.DataFrame()
mix_train['text'] = X_trainmix
mix_train['emotions'] = y_trainmix

mix_test = pd.DataFrame()
mix_test['text'] = X_valmix
mix_test['emotions'] = y_valmix

In [15]:
print(X_trainmix)

15092    My Twitteeeerrr babiieshow Ive missed you  I h...
2132     iTunes upgrade reset playcount to zero  How wi...
15270    Maybe i won't go for a run .. i have no runnin...
20054    @DonnieWahlberg sounds like you all had a grea...
33079    kill bill pt 2 .....I love em both wish they w...
                               ...                        
24749    Waking up at unGodly hours to go to work is st...
3661     @JamanthaLopez it was me the whole time I did ...
8892     @cdlstatus are you guys dead again?  I can't l...
25238             @mikedignammusic coool thanks a lot  xxx
39476             @sojanjose4u it's working now,IT'S OKAY!
Name: text, Length: 35764, dtype: object


In [16]:
#cheeck no overlap
count = 0
l1 = X_trainmix.to_list()
l2 = X_valmix.to_list()
for i in l2:
  if i in l1:
    count += 1

print(count)

0


In [17]:
mix_train.count()

text        35764
emotions    35764
dtype: int64

In [18]:
cur_training = pd.read_csv("train_data_imbalanced_stratified_no_dupe_136k_40k.csv")

cur_training.shape

(136657, 3)

In [19]:
cur_training.head()

cur_training = cur_training.drop(['Unnamed: 0'], axis=1)
cur_training = cur_training.drop_duplicates('text', keep=False)

In [20]:
cur_training.shape

(136657, 2)

In [21]:
merged_train = pd.concat([cur_training, mix_train])
#merged_train = merged_train.drop(['Unnamed: 0'], axis=1)
merged_train = merged_train.drop_duplicates('text', keep=False)

In [22]:
count = 0
l1 = cur_training['text'].to_list()
l2 = merged_train['text'].to_list()
for i in l1:
  if i in l2:
    count+=1
print(count)

133882


In [23]:
#this is with 53 k training record
merged_train.count()

text        166871
emotions    166871
dtype: int64

In [24]:
merged_train = merged_train.sample(frac=1)

In [26]:
merged_train.count()

text        166871
emotions    166871
dtype: int64

In [28]:
## this is with 30 k training
#X_trainmix_1, X_valmix_1, y_trainmix_1, y_valmix_1= data_splitter(to_mix, test_pct=0.5, train_pct=0.5)
#len(X_trainmix_1)

In [29]:
#mix_smaller = pd.DataFrame()
#mix_smaller['text'] = X_trainmix_1
#mix_smaller['emotions'] = y_trainmix_1

In [30]:
#X_trainmix_1

In [31]:
#count = 0
#x1 = mix_smaller['text'].to_list()
#x2 = cur_training['text'].to_list()
#for i in x1:
#  if i in x2:
#    count+=1
#print(count)



In [32]:
#merged_train_smaller = pd.concat([cur_training, mix_smaller])
#merged_train_smaller = merged_train_smaller.drop(['Unnamed: 0'], axis=1)
#merged_train_smaller = merged_train_smaller.drop_duplicates('text', keep=False)

In [33]:
#count = 0
#x1 = mix_smaller['text'].to_list()
#x2 = merged_train_smaller['text'].to_list()
#for i in x1:
#  if i in x2:
#    count+=1
#print(count)

In [34]:
#merged_train_smaller.count()
#merged_train_smaller = merged_train_smaller.sample(frac=1)


In [65]:
from google.colab import files
#merged_train.to_csv('training_52k_mixed_clean_kaggle_230401.csv')
#files.download('training_52k_mixed_clean_kaggle_230401.csv')

In [75]:
#merged_train_smaller.to_csv('training_32k_mixed_clean_kaggle_230401.csv')
#files.download('training_32k_mixed_clean_kaggle_230401.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
#cur_test = pd.read_csv("test_data_imbalanced_stratified_no_dupe.csv")
#cur_test = cur_test.drop_duplicates("text", keep=False)

In [36]:
#cur_test.shape

In [37]:
#mix_test = mix_test.drop_duplicates("text", keep=False)

In [38]:
#mix_test.shape

In [39]:
#joined_test = pd.concat([cur_test, mix_test])
#joined_test.count()

In [40]:
#joined_test = joined_test.drop_duplicates("text", keep=False)
#joined_test.count()

In [41]:
#joined_test = joined_test.drop('Unnamed: 0', axis= 1)

In [42]:
#count = 0
#x1 = cur_test['text'].to_list()
#x2 = mix_test['text'].to_list()
#for i in x1:
#  if i in x2:
#    count+=1
#print(count)

In [44]:
def get_proportions(data):
  # get the value counts of each unique value
  counts = data.value_counts()

  # convert the counts to percentages
  percentages = counts / counts.sum() * 100

  return pd.DataFrame({'count': counts, 'percentage': percentages})  

#get_proportions(joined_test['emotions'])

In [45]:
get_proportions(merged_train['emotions'])

,count,percentage
joy,54793,32.835544
sadness,47079,28.212811
fear,21819,13.075370
anger,20103,12.047030
love,12153,7.282871
surprise,5495,3.292963
neutral,5429,3.253411


In [46]:
#count = 0
#x1 = merged_train['text'].to_list()
#x2 = joined_test['text'].to_list()
#for i in x1:
#  if i in x2:
#    count+=1
#print(count)

In [47]:
print(merged_train.shape)
#print(joined_test.shape)
#print(merged_train_smaller.shape)

(166871, 2)


In [48]:
#totality = pd.concat([merged_train, joined_test])
totality = merged_train
totality.shape

(166871, 2)

In [49]:
totality = totality.drop_duplicates('text', keep=False)
totality.shape

(166871, 2)

In [183]:
#totality1 = pd.concat([merged_train_smaller, joined_test])
#totality1.shape

(78326, 2)

In [184]:
#totality1 = totality1.drop_duplicates('text', keep=False)
#totality1.shape

(77394, 2)

In [51]:
train_50k = totality.iloc[0:50000]
test_30k = totality.iloc[50001:80000]
val_10k = totality.iloc[80001:90000]

In [56]:
count = 0
x1 = train_50k['text'].to_list()
x2 = test_30k['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)

0


In [60]:
count = 0
x1 = train_50k['text'].to_list()
x2 = val_10k['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)


count = 0
x1 = test_30k['text'].to_list()
x2 = val_10k['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)

0
0


In [67]:
print(train_50k.shape)
print(val_10k.shape)
print(test_30k.shape)

(50000, 2)
(9999, 2)
(29999, 2)


In [52]:
train_20k = totality.iloc[0:20000]
test_30k_1 = totality.iloc[20001:50000]
val_10k_1 = totality.iloc[50001:60000]

In [57]:
count = 0
x1 = train_20k['text'].to_list()
x2 = test_30k_1['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)

0


In [61]:
count = 0
x1 = val_10k_1['text'].to_list()
x2 = test_30k_1['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)

count = 0
x1 = val_10k_1['text'].to_list()
x2 = train_20k['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)

0
0


In [68]:
print(train_20k.shape)
print(val_10k_1.shape)
print(test_30k_1.shape)

(20000, 2)
(9999, 2)
(29999, 2)


In [53]:
train_80k = totality.iloc[0:80000]
test_30k_2 = totality.iloc[80001:110000]
val_10k_2 = totality.iloc[110001:120000]

In [58]:
count = 0
x1 = train_80k['text'].to_list()
x2 = test_30k_2['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)

0


In [62]:
count = 0
x1 = val_10k_2['text'].to_list()
x2 = test_30k_2['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)

count = 0
x1 = val_10k_2['text'].to_list()
x2 = train_80k['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)

0
0


In [69]:
print(train_80k.shape)
print(val_10k_2.shape)
print(test_30k_2.shape)

(80000, 2)
(9999, 2)
(29999, 2)


In [54]:
train_100k = totality.iloc[0:100000]
test_30k_3 = totality.iloc[100001:130000]
val_10k_3 = totality.iloc[130001:140000]

In [59]:
count = 0
x1 = train_100k['text'].to_list()
x2 = test_30k_3['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)

0


In [63]:
count = 0
x1 = val_10k_3['text'].to_list()
x2 = train_100k['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)

count = 0
x1 = val_10k_3['text'].to_list()
x2 = test_30k_3['text'].to_list()
for i in x1:
  if i in x2:
    count+=1
print(count)

0
0


In [70]:
print(train_100k.shape)
print(val_10k_3.shape)
print(test_30k_3.shape)

(100000, 2)
(9999, 2)
(29999, 2)


In [66]:
train_50k.to_csv('train_50k_10k_30k_mixed_clean_kaggle_230402.csv')
test_30k.to_csv('test_50k_10k_30k_mixed_clean_kaggle_230402.csv')
val_10k.to_csv('val_50k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('train_50k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('test_50k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('val_50k_10k_30k_mixed_clean_kaggle_230402.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
train_20k.to_csv('train_20k_10k_30k_mixed_clean_kaggle_230402.csv')
test_30k_1.to_csv('test_20k_10k_30k_mixed_clean_kaggle_230402.csv')
val_10k_1.to_csv('val_20k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('train_20k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('test_20k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('val_20k_10k_30k_mixed_clean_kaggle_230402.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [72]:
train_80k.to_csv('train_80k_10k_30k_mixed_clean_kaggle_230402.csv')
test_30k_2.to_csv('test_80k_10k_30k_mixed_clean_kaggle_230402.csv')
val_10k_2.to_csv('val_80k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('train_80k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('test_80k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('val_80k_10k_30k_mixed_clean_kaggle_230402.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [73]:
train_100k.to_csv('train_100k_10k_30k_mixed_clean_kaggle_230402.csv')
test_30k_3.to_csv('test_100k_10k_30k_mixed_clean_kaggle_230402.csv')
val_10k_3.to_csv('val_100k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('train_100k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('test_100k_10k_30k_mixed_clean_kaggle_230402.csv')
files.download('val_100k_10k_30k_mixed_clean_kaggle_230402.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>